## Simple web search with phidata and Ollama

This script demonstrates the creation and usage of a Web Agent that leverages Ollama with a small language model and the DuckDuckGo search tool to respond to user queries.
Let's break down the key components:


## Overview
First we'll import all the necessary packages and modules:


In [1]:
from phi.agent import Agent
from phi.model.ollama import OllamaTools
from phi.tools.duckduckgo import DuckDuckGo

We are using the OllamaTools module that supports function calling via XML tags. A workaround that works with most models and doesn't need "native function-calling" support. We also import the DuckDuckGo tool that provides web and news search functions.

Now we are defining an agent, but first we will define the Ollama host to either run Ollama locally or call a remote Ollama server:

In [2]:
OLLAMA_HOST="localhost"

In [3]:
simple_agent = Agent(
    name="Simple Agent",
    model=OllamaTools(id="hermes3:8b-llama3.1-q8_0", host=OLLAMA_HOST),
    markdown=True,
)

In [4]:
simple_agent.print_response("What is the latest news from Darmstadt?", stream=True)

Output()

As you can see sometimes the model already knows quite a bit about Darmstadt, or think it does as it might hallucinate. We can try to avoid that but there's no guarantee that this will work.

In [5]:
simple_agent2 = Agent(
    name="Simple Agent 2",
    model=OllamaTools(id="hermes3:8b-llama3.1-q8_0", host=OLLAMA_HOST),
    markdown=True,
    prevent_hallucinations=True,
    debug_mode=True
)

DEBUG    Debug logs enabled

In [6]:
simple_agent2.print_response("What is the latest news from Darmstadt?", stream=True)

DEBUG    *********** Agent Run Start: 3e70123e-369b-462a-be14-27ca77948720 ***********

DEBUG    ---------- Ollama Response Start ----------

DEBUG    ============== system ==============

DEBUG    ## Instructions                                                                                           
         - **Do not make up information:** If you don't know the answer or cannot determine from the context       
         provided, say 'I don't know'.                                                                             
         - Use markdown to format your answers.

DEBUG    ============== user ==============

...

DEBUG    ---------- Ollama Response End ----------

DEBUG    Added 2 Messages to AgentMemory

DEBUG    Added AgentRun to AgentMemory

DEBUG    --**-- Logging Agent Run

DEBUG    *********** Agent Run End: 3e70123e-369b-462a-be14-27ca77948720 ***********

This output already looks much better than before but the best method to avoid hallucinations is to provide the LLM with a tool to search the web for information. Caveat: This doesn't guarantee that the information will be correct 😉

In [7]:
web_agent = Agent(
    name="Web Agent",
    #model=OllamaTools(id="qwen2.5:7b-instruct-q8_0", host=OLLAMA_HOST),
    #model=OllamaTools(id="qwen2.5:3b-instruct-q8_0", host=OLLAMA_HOST),
    #model=OllamaTools(id="llama3.2:3b-instruct-q8_0", host=OLLAMA_HOST),
    #model=OllamaTools(id="smollm2:1.7b-instruct-q8_0", host=OLLAMA_HOST), # too smol, can't search
    #model=OllamaTools(id="granite3-dense:2b-instruct-q8_0", host=OLLAMA_HOST),
    model=OllamaTools(id="hermes3:8b-llama3.1-q8_0", host=OLLAMA_HOST),
    tools=[DuckDuckGo()],
    show_tool_calls=True,
    markdown=True,
)

DEBUG    Function: duckduckgo_search registered with duckduckgo

DEBUG    Function: duckduckgo_news registered with duckduckgo

DEBUG    *********** Agent ID: 542498dc-52f7-4df9-8c47-f677ca8639d9 ***********

DEBUG    *********** Session ID: bd6f0ee4-0253-484b-8530-8ec19f74570f ***********

We'll use the Hermes 3 8B model fine-tuned on Llama 3.1 with Q8 quantization. This model has worked best in our tests with tools/function calling. Feel free to try smaller models like the 3B Qwen 2.5 or Llama 3.2. These smaller models might not work all the time.

Now that the agent is defined and has access to DuckDuckGo web search we can ask it about what's happening in Darmstadt. We set `show_tool_calls` to `True` to see which tools the agent is calling with which parameters.

In [8]:
web_agent.print_response("What is the latest news from Darmstadt?", stream=True)

Output()

Congratulations, you have created your first basic agent with web search.